# LGBM Optuna Optimizer

## For Kaggle AMEX default prediction competition data

https://www.kaggle.com/competitions/amex-default-prediction/data

This competition had some random looking data where no information on columns was given. The notebook is mainly to allow me to copy it for use in LGBM optimization in other tabular data cases.

This assumes LGBM  and various other Python libraries are installed. And GPU configured. I used a Docker image and Pipfile with all these installed.


In [1]:
#!pip install lightgbm


In [2]:
import pandas as pd
import numpy as np

import optuna  # pip install optuna
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from optuna.integration import LightGBMPruningCallback
from sklearn.model_selection import train_test_split
import lightgbm as lgbm
from tqdm import tqdm
import gc
from sklearn import preprocessing

tqdm.pandas()

import time

N_FOLDS = 5

In [3]:
# https://www.kaggle.com/kyakovlev
# https://www.kaggle.com/competitions/amex-default-prediction/discussion/327534
def amex_metric_mod(y_true, y_pred):

    labels     = np.transpose(np.array([y_true, y_pred]))
    labels     = labels[labels[:, 1].argsort()[::-1]]
    weights    = np.where(labels[:,0]==0, 20, 1)
    cut_vals   = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four   = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])

    gini = [0,0]
    for i in [1,0]:
        labels         = np.transpose(np.array([y_true, y_pred]))
        labels         = labels[labels[:, i].argsort()[::-1]]
        weight         = np.where(labels[:,0]==0, 20, 1)
        weight_random  = np.cumsum(weight / np.sum(weight))
        total_pos      = np.sum(labels[:, 0] *  weight)
        cum_pos_found  = np.cumsum(labels[:, 0] * weight)
        lorentz        = cum_pos_found / total_pos
        gini[i]        = np.sum((lorentz - weight_random) * weight)

    return 0.5 * (gini[1]/gini[0] + top_four)


In [4]:
def format_time(seconds):
    seconds = int(seconds)
    minutes = seconds // 60
    hours = minutes // 60
    minutes = minutes % 60
    seconds = seconds % 60
    if hours > 0:
        return f"{hours}h, {minutes}m, {seconds}s"
    if minutes > 0:
        return f"{minutes}m, {seconds}s"
    return f"{seconds}s"

In [5]:
import pickle

def pickle_save(to_save, filename):
    with open(f'{filename}.pickle','wb') as f:
        pickle.dump(to_save,f)
    

# Load Data

In [6]:
df_train = pd.read_parquet("train_lagged.parquet", engine="pyarrow")
df_train.head()

,customer_ID,S_2_first,S_2_mean,S_2_std,S_2_min,S_2_max,S_2_last,P_2_first,P_2_mean,P_2_std,...,D_66_count,D_66_first,D_66_last,D_66_nunique,D_68_count,D_68_first,D_68_last,D_68_nunique,target,fake_splitter
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0,0.0,0.0,0,0,0,0.94,0.933846,0.024337,...,13,0,0,1,13,7,7,1,0,True
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0,0.0,0.0,0,0,0,0.93,0.899231,0.022899,...,13,0,0,1,13,7,7,1,0,True
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0,0.0,0.0,0,0,0,0.88,0.877692,0.027735,...,13,0,0,1,13,7,7,1,0,True
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0,0.0,0.0,0,0,0,0.57,0.597692,0.018777,...,13,0,0,1,13,3,4,3,0,True
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0,0.0,0.0,0,0,0,0.94,0.893077,0.042502,...,13,2,2,1,13,7,7,1,0,True


In [7]:
#df_train["target"] > 0.6
strat = df_train["fake_splitter"]
df_train = df_train.drop("fake_splitter", axis=1)

In [8]:
df_test = pd.read_parquet("test_lagged.parquet", engine="pyarrow")
df_test.head()

,customer_ID,S_2_first,S_2_mean,S_2_std,S_2_min,S_2_max,S_2_last,P_2_first,P_2_mean,P_2_std,...,D_64_last,D_64_nunique,D_66_count,D_66_first,D_66_last,D_66_nunique,D_68_count,D_68_first,D_68_last,D_68_nunique
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0,0.0,0.0,0,0,0,0.63,0.602222,0.019861,...,4,2,9,0,0,1,9,0,7,2
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0,0.0,0.0,0,0,0,0.89,0.860769,0.030947,...,1,1,13,0,0,1,13,7,7,1
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0,0.0,0.0,0,0,0,0.77,0.748462,0.061893,...,4,2,13,2,2,1,13,7,5,2
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0,0.0,0.0,0,0,0,0.51,0.474615,0.026962,...,3,1,13,0,0,1,13,6,6,1
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0,0.0,0.0,0,0,0,0.29,0.323846,0.050751,...,3,2,13,0,0,1,13,7,6,2


In [11]:
def find_new_cat_cols(df_from):
    new_cat_cols = []
    for cat_col in cat_cols:
        new_cat_cols.extend([col for col in df_from.columns if col.startswith(cat_col)])
    return new_cat_cols



In [12]:
cat_cols = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68'] 
cat_cols = find_new_cat_cols(df_train)

# LGBM Optuna Optimizer

In [37]:
optuna_eval_results = {}
optuna_iteration_index = 0
optuna_losses = []
optuna_accuracies = []
optuna_amexes = []
optuna_parameters = []
optuna_seconds = []

def optuna_objective(trial, X, y, strat, verbosity=100):
    global optuna_iteration_index
    optuna_iteration_index += 1
    #param_grid = {}  # to be filled in later
    param_grid = {
        "verbosity": -1,
        "n_estimators": trial.suggest_categorical("n_estimators", [12000]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
        #GPU only allows max bin 255 on LGBM
        "max_bin": trial.suggest_int("max_bin", 100, 255),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": None,
        "bagging_freq": None,
        "feature_fraction": trial.suggest_float(
            "feature_fraction", 0.2, 0.95, step=0.05
        ),
        'subsample': None,  # overridden by bagging_fraction
        'reg_alpha': None,  # overridden by lambda_l1
        'reg_lambda': None,  # overridden by lambda_l2
        'min_sum_hessian_in_leaf': None,  # overrides min_child_weight
        'min_child_samples': None,  # overridden by min_data_in_leaf
        'colsample_bytree': None,  # overridden by feature_fraction
        'subsample_freq': None, # bagging_freq
        'min_split_gain': None, # 
        #'min_split_gain': None, # min_gain_to_split

#        'boosting_type': trial.suggest_categorical('boosting_type',['dart']), #only try DART now
        'boosting_type': trial.suggest_categorical('boosting_type',['gbdt', 'goss', 'dart']), #no DART atm
    }
    if param_grid["boosting_type"] != "goss":
        param_grid["bagging_fraction"] = trial.suggest_float(
            "bagging_fraction", 0.2, 0.95, step=0.05
        )
        param_grid["bagging_freq"] = trial.suggest_categorical("bagging_freq", [1, 5, 10])

    if param_grid["boosting_type"] == "dart":
        param_grid["n_estimators"] = 2000 #TODO: maybe less in search, more in actual selected train
        
        
    print(f"STARTING OPTIMIZATION iteration {optuna_iteration_index}, data size: {X.shape}, params={param_grid}")
    #split data into N pieces, run the optimizer on only N_FOLDS_RUN of them for speed
    N_FOLDS_RUN = N_FOLDS
    cv = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=69)
#    cv = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=1121218)

    cv_scores = np.empty(N_FOLDS_RUN)
    cv_accuracies = np.empty(N_FOLDS_RUN)
    cv_amexes = np.empty(N_FOLDS_RUN)
    
    time_start_all_folds = time.time()
    acc_score_total = 0
    n_classes = 1
    train_preds = np.zeros((X.shape[0], n_classes))
    amex_total = 0
    
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, strat)):
        time_start_this_fold = time.time()
        print(f"=== STARTING FOLD {idx+1}/{N_FOLDS_RUN} ({N_FOLDS}) ===")
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        print("creating classifier")
        model = lgbm.LGBMClassifier(objective="binary", **param_grid, device="gpu")
        print("fitting")
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_test, y_test)],
            eval_metric="binary_logloss",
            categorical_feature=cat_cols,
            #early_stopping_rounds=100,
            callbacks=[
                lgbm.early_stopping(100, verbose = False), 
                lgbm.record_evaluation(optuna_eval_results),
                lgbm.log_evaluation(verbosity) #by default every round
                #LightGBMPruningCallback(trial, "binary_logloss")
            ],  # Add a pruning callback
        )

        print("predicting")

        #best iteration used by default https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html#lightgbm.LGBMClassifier.predict_proba
        preds = model.predict_proba(X_test)

        preds_true_amex = preds[:, 1]
        preds_true = preds[:, 1].flatten()
        preds_true.shape = (y_test.shape[0], 1)
        amex_y_test = np.array(y_test)
        amex_y_test.shape = (y_test.shape[0], 1)
        np.add.at(train_preds, test_idx, preds_true)
        print(f"count zerO: {np.count_nonzero(train_preds)}")

        preds_this_round = (preds >= 0.5)[:,1]

        acc_score = accuracy_score(y_test, preds_this_round)
        acc_score_total += acc_score

        amex_score = amex_metric_mod(y_test, preds_true_amex)
        amex_total += amex_score

        log_loss_fold = log_loss(y_test, preds)
        cv_scores[idx] = log_loss_fold
        cv_accuracies[idx] = acc_score
        cv_amexes[idx] = amex_score
        print(f"amex: {amex_score}")

        print(f"=== FINISHED FOLD {idx+1}/{N_FOLDS_RUN} ({N_FOLDS}), log loss={log_loss_fold}, accuracy={acc_score}, amex={amex_score} ===")
        time_total_this_fold = time.time() - time_start_this_fold
        print(f"time to run this fold: {format_time(time_total_this_fold)}")
        
        if idx+1 >= N_FOLDS_RUN:
            break
            
        if param_grid["boosting_type"] == "dart":
            if (amex_score < 0.79 and idx == 0) or (amex_score < 0.78 and idx > 0):
                print("low score on amex, dropping loop")
                return amex_score
        
    time_total_all_folds = time.time() - time_start_all_folds        
    print(f"time to run folds: {format_time(time_total_all_folds)}")
    
    iteration_log_loss = np.mean(cv_scores)
    iteration_accuracy = np.mean(cv_accuracies)
    iteration_amex = np.mean(cv_amexes)
    optuna_losses.append(iteration_log_loss)
    optuna_amexes.append(iteration_amex)
    optuna_accuracies.append(iteration_accuracy)
    optuna_parameters.append(param_grid)
    optuna_seconds.append(int(time_total_all_folds))
    print(f"iteration logloss: {iteration_log_loss}, iteration acc: {iteration_accuracy}, Iteration amex: {iteration_amex}")
    return iteration_amex #iteration_log_loss

#TODO: use LGBM's unbalanced target weighting to try if it has any effect

# Train and Predict after Finding Optimized Params

In [10]:
def train_and_predict(optuna_params, df_from, strat):
    X = df_from.drop(["customer_ID", "target"], axis=1)
    y = df_from["target"]

    verbosity=100 #print status every n iteration of tree building
    optuna_eval_results = {}

    cv = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=1121218)

    cv_scores = np.empty(N_FOLDS)
    cv_accuracies = np.empty(N_FOLDS)
    time_start_all_folds = time.time()
    acc_score_total = 0
    amex_score_total = 0
    models = []

    n_classes = 1
    train_preds = np.zeros((X.shape[0], n_classes))
    #balanced_preds = np.empty([X.shape[0], n_classes], dtype = float)

    for idx, (train_idx, test_idx) in enumerate(cv.split(X, strat)):
    #    print(idx,train_idx,test_idx)
        time_start_this_fold = time.time()
        print(f"=== STARTING FOLD {idx+1}/{N_FOLDS} ===")
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        print("creating classifier")
    #    model = lgbm.LGBMClassifier(objective="binary", categorical_feature=cat_cols, **optuna_parameters_balanced[best_idx])
        model = lgbm.LGBMClassifier(objective="binary", **optuna_params, device="gpu")
        print("fitting")
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_test, y_test)],
            eval_metric="binary_logloss",
            categorical_feature=cat_cols,
            #early_stopping_rounds=100,
            callbacks=[
                lgbm.early_stopping(100, verbose = False), 
                lgbm.record_evaluation(optuna_eval_results),
                lgbm.log_evaluation(verbosity) #by default every round
                #LightGBMPruningCallback(trial, "binary_logloss")
            ],  # Add a pruning callback
        )
        models.append(model)

        print("predicting")
        preds = model.predict_proba(X_test)
        preds_true = preds[:, 1].flatten()
        preds_true.shape = (y_test.shape[0], 1)
        np.add.at(train_preds, test_idx, preds_true)
        print(f"count zerO: {np.count_nonzero(train_preds)}")

        preds_true_amex = preds[:, 1]
        preds_this_round = (preds >= 0.5)[:,1]
    #        positives=(preds >= 0.5)[:,1]
    #        print()
        acc_score = accuracy_score(y_test, preds_this_round)
        acc_score_total += acc_score

        amex_score = amex_metric_mod(y_test, preds_true_amex)
        amex_score_total += amex_score

        log_loss_fold = log_loss(y_test, preds)
        cv_scores[idx] = log_loss_fold
        cv_accuracies[idx] = acc_score
        print(f"=== FINISHED FOLD {idx+1}/{N_FOLDS}, log loss={log_loss_fold}, accuracy={acc_score}, amex={amex_score} ===")
        time_total_this_fold = time.time() - time_start_this_fold
        print(f"time to run this fold: {format_time(time_total_this_fold)}")

    print(f"amex avg: {amex_score_total/N_FOLDS}")
    return models, train_preds


# Split Data and Run Optimizer Search

In [13]:
X = df_train.drop(["customer_ID", "target"], axis=1)
y = df_train["target"]
stratzip = zip(strat, y)
#stratified k-fold only supports single binary value, tuple stratification breaks it
stratzip = [f"{a}-{b}" for (a,b) in stratzip]
#stratzip = list(stratzip)


In [14]:
optuna_eval_results = {}
optuna_iteration_index = 0
optuna_losses = []
optuna_accuracies = []
optuna_parameters = []
optuna_seconds = []

study = optuna.create_study(direction="maximize", study_name="LGBM Classifier")
func = lambda trial: optuna_objective(trial, X, y, stratzip)
study.optimize(func, n_trials=20) #TODO: set to 50-100 for real search, just protoing now and 20 is faster

[I 2022-09-09 10:16:32,894] A new study created in memory with name: LGBM Classifier


STARTING OPTIMIZATION iteration 1, data size: (458913, 1580), params={'verbosity': -1, 'n_estimators': 12000, 'learning_rate': 0.03680792839869354, 'num_leaves': 2040, 'max_depth': 4, 'min_data_in_leaf': 300, 'max_bin': 157, 'lambda_l1': 90, 'lambda_l2': 85, 'min_gain_to_split': 8.707453098189157, 'bagging_fraction': None, 'bagging_freq': None, 'feature_fraction': 0.45, 'subsample': None, 'reg_alpha': None, 'reg_lambda': None, 'min_sum_hessian_in_leaf': None, 'min_child_samples': None, 'colsample_bytree': None, 'subsample_freq': None, 'min_split_gain': None, 'boosting_type': 'goss'}
=== STARTING FOLD 1/5 (5) ===
creating classifier
fitting
[100]	valid_0's binary_logloss: 0.246895
[200]	valid_0's binary_logloss: 0.232113
[300]	valid_0's binary_logloss: 0.227489
[400]	valid_0's binary_logloss: 0.225009
[500]	valid_0's binary_logloss: 0.223552
[600]	valid_0's binary_logloss: 0.222639
[700]	valid_0's binary_logloss: 0.222075
[800]	valid_0's binary_logloss: 0.221662
[900]	valid_0's binary_l

[5400]	valid_0's binary_logloss: 0.218653
[5500]	valid_0's binary_logloss: 0.218653
predicting
count zerO: 275349
amex: 0.7900593995414786
=== FINISHED FOLD 3/5 (5), log loss=0.21865099606998023, accuracy=0.9028687229661266, amex=0.7900593995414786 ===
time to run this fold: 4m, 44s
=== STARTING FOLD 4/5 (5) ===
creating classifier
fitting
[100]	valid_0's binary_logloss: 0.247258
[200]	valid_0's binary_logloss: 0.231936
[300]	valid_0's binary_logloss: 0.226798
[400]	valid_0's binary_logloss: 0.224119
[500]	valid_0's binary_logloss: 0.222477
[600]	valid_0's binary_logloss: 0.221409
[700]	valid_0's binary_logloss: 0.220711
[800]	valid_0's binary_logloss: 0.220253
[900]	valid_0's binary_logloss: 0.219922
[1000]	valid_0's binary_logloss: 0.219646
[1100]	valid_0's binary_logloss: 0.219461
[1200]	valid_0's binary_logloss: 0.219318
[1300]	valid_0's binary_logloss: 0.219227
[1400]	valid_0's binary_logloss: 0.219116
[1500]	valid_0's binary_logloss: 0.219023
[1600]	valid_0's binary_logloss: 0.21

[I 2022-09-09 10:40:57,417] Trial 0 finished with value: 0.7899761091944011 and parameters: {'n_estimators': 12000, 'learning_rate': 0.03680792839869354, 'num_leaves': 2040, 'max_depth': 4, 'min_data_in_leaf': 300, 'max_bin': 157, 'lambda_l1': 90, 'lambda_l2': 85, 'min_gain_to_split': 8.707453098189157, 'feature_fraction': 0.45, 'boosting_type': 'goss'}. Best is trial 0 with value: 0.7899761091944011.


count zerO: 458913
amex: 0.7850556908956438
=== FINISHED FOLD 5/5 (5), log loss=0.22032579777273437, accuracy=0.9036630276088994, amex=0.7850556908956438 ===
time to run this fold: 5m, 38s
time to run folds: 24m, 24s
iteration logloss: 0.21884749788881802, iteration acc: 0.9035612414241146, Iteration amex: 0.7899761091944011
STARTING OPTIMIZATION iteration 2, data size: (458913, 1580), params={'verbosity': -1, 'n_estimators': 12000, 'learning_rate': 0.22077379374058514, 'num_leaves': 2060, 'max_depth': 12, 'min_data_in_leaf': 7700, 'max_bin': 209, 'lambda_l1': 90, 'lambda_l2': 20, 'min_gain_to_split': 3.925552320660292, 'bagging_fraction': None, 'bagging_freq': None, 'feature_fraction': 0.7, 'subsample': None, 'reg_alpha': None, 'reg_lambda': None, 'min_sum_hessian_in_leaf': None, 'min_child_samples': None, 'colsample_bytree': None, 'subsample_freq': None, 'min_split_gain': None, 'boosting_type': 'goss'}
=== STARTING FOLD 1/5 (5) ===
creating classifier
fitting
[100]	valid_0's binary_l

...

count zerO: 458913
amex: 0.7883531092475912
=== FINISHED FOLD 5/5 (5), log loss=0.21830215181657397, accuracy=0.903880935259637, amex=0.7883531092475912 ===
time to run this fold: 2m, 27s
time to run folds: 12m, 25s
iteration logloss: 0.21659017470205644, iteration acc: 0.9045875789804807, Iteration amex: 0.7930696238566487
STARTING OPTIMIZATION iteration 19, data size: (458913, 1580), params={'verbosity': -1, 'n_estimators': 12000, 'learning_rate': 0.2530154318515711, 'num_leaves': 1140, 'max_depth': 6, 'min_data_in_leaf': 6500, 'max_bin': 190, 'lambda_l1': 55, 'lambda_l2': 10, 'min_gain_to_split': 5.3843132626962475, 'bagging_fraction': None, 'bagging_freq': None, 'feature_fraction': 0.6000000000000001, 'subsample': None, 'reg_alpha': None, 'reg_lambda': None, 'min_sum_hessian_in_leaf': None, 'min_child_samples': None, 'colsample_bytree': None, 'subsample_freq': None, 'min_split_gain': None, 'boosting_type': 'goss'}
=== STARTING FOLD 1/5 (5) ===
creating classifier
fitting
[100]	vali

[I 2022-09-09 12:36:39,724] Trial 18 finished with value: 0.7858538314512433 and parameters: {'n_estimators': 12000, 'learning_rate': 0.2530154318515711, 'num_leaves': 1140, 'max_depth': 6, 'min_data_in_leaf': 6500, 'max_bin': 190, 'lambda_l1': 55, 'lambda_l2': 10, 'min_gain_to_split': 5.3843132626962475, 'feature_fraction': 0.6000000000000001, 'boosting_type': 'goss'}. Best is trial 2 with value: 0.7932202487793407.


count zerO: 458913
amex: 0.7799241803066352
=== FINISHED FOLD 5/5 (5), log loss=0.2243157882712578, accuracy=0.9007539604715522, amex=0.7799241803066352 ===
time to run this fold: 36s
time to run folds: 3m, 40s
iteration logloss: 0.22191976487962303, iteration acc: 0.9017787663377262, Iteration amex: 0.7858538314512433
STARTING OPTIMIZATION iteration 20, data size: (458913, 1580), params={'verbosity': -1, 'n_estimators': 12000, 'learning_rate': 0.09469158229493857, 'num_leaves': 1840, 'max_depth': 7, 'min_data_in_leaf': 4000, 'max_bin': 177, 'lambda_l1': 20, 'lambda_l2': 35, 'min_gain_to_split': 2.823134449326043, 'bagging_fraction': 0.35000000000000003, 'bagging_freq': 5, 'feature_fraction': 0.55, 'subsample': None, 'reg_alpha': None, 'reg_lambda': None, 'min_sum_hessian_in_leaf': None, 'min_child_samples': None, 'colsample_bytree': None, 'subsample_freq': None, 'min_split_gain': None, 'boosting_type': 'gbdt'}
=== STARTING FOLD 1/5 (5) ===
creating classifier
fitting
[100]	valid_0's b

[I 2022-09-09 12:39:04,007] Trial 19 finished with value: 0.7889612678747275 and parameters: {'n_estimators': 12000, 'learning_rate': 0.09469158229493857, 'num_leaves': 1840, 'max_depth': 7, 'min_data_in_leaf': 4000, 'max_bin': 177, 'lambda_l1': 20, 'lambda_l2': 35, 'min_gain_to_split': 2.823134449326043, 'feature_fraction': 0.55, 'boosting_type': 'gbdt', 'bagging_fraction': 0.35000000000000003, 'bagging_freq': 5}. Best is trial 2 with value: 0.7932202487793407.


count zerO: 458913
amex: 0.7848318326750809
=== FINISHED FOLD 5/5 (5), log loss=0.2214314717300738, accuracy=0.9028785600662439, amex=0.7848318326750809 ===
time to run this fold: 30s
time to run folds: 2m, 24s
iteration logloss: 0.21950076865634038, iteration acc: 0.9032801425014754, Iteration amex: 0.7889612678747275


# Check Optimizer Results

In [15]:
df_params = pd.DataFrame(optuna_parameters)
df_params["loss"] = optuna_losses
df_params["accuracy"] = optuna_accuracies
df_params["amex"] = optuna_amexes
df_params["time"] = optuna_seconds


In [16]:
df_params.to_csv("optimizer_results_lgbm_lagged.csv")

In [17]:
df_params = pd.read_csv("optimizer_results_lgbm_lagged.csv")

In [18]:
df_params.head(5)

,Unnamed: 0,verbosity,n_estimators,learning_rate,num_leaves,max_depth,min_data_in_leaf,max_bin,lambda_l1,lambda_l2,...,min_sum_hessian_in_leaf,min_child_samples,colsample_bytree,subsample_freq,min_split_gain,boosting_type,loss,accuracy,amex,time
0,0,-1,12000,0.036808,2040,4,300,157,90,85,...,NaN,NaN,NaN,NaN,NaN,goss,0.218847,0.903561,0.789976,1464
1,1,-1,12000,0.220774,2060,12,7700,209,90,20,...,NaN,NaN,NaN,NaN,NaN,goss,0.221447,0.902304,0.786098,289
2,2,-1,2000,0.252294,620,10,7500,146,25,0,...,NaN,NaN,NaN,NaN,NaN,dart,0.216523,0.904618,0.793220,689
3,3,-1,12000,0.155771,1440,3,5900,251,40,80,...,NaN,NaN,NaN,NaN,NaN,gbdt,0.219392,0.903197,0.789396,103
4,4,-1,12000,0.139857,2580,11,800,190,45,60,...,NaN,NaN,NaN,NaN,NaN,gbdt,0.221637,0.902018,0.786261,80


In [19]:
df_params.sort_values(by="amex", ascending=False).head(10)

,Unnamed: 0,verbosity,n_estimators,learning_rate,num_leaves,max_depth,min_data_in_leaf,max_bin,lambda_l1,lambda_l2,...,min_sum_hessian_in_leaf,min_child_samples,colsample_bytree,subsample_freq,min_split_gain,boosting_type,loss,accuracy,amex,time
2,2,-1,2000,0.252294,620,10,7500,146,25,0,...,NaN,NaN,NaN,NaN,NaN,dart,0.216523,0.904618,0.793220,689
13,13,-1,2000,0.251956,1100,7,6900,185,20,10,...,NaN,NaN,NaN,NaN,NaN,dart,0.216590,0.904588,0.793070,745
9,9,-1,2000,0.197107,740,9,8100,116,15,20,...,NaN,NaN,NaN,NaN,NaN,dart,0.216647,0.904496,0.793006,797
8,8,-1,2000,0.215375,40,8,10000,133,10,5,...,NaN,NaN,NaN,NaN,NaN,dart,0.217503,0.904004,0.792391,780
12,12,-1,2000,0.179207,600,9,4000,172,60,35,...,NaN,NaN,NaN,NaN,NaN,dart,0.217463,0.904054,0.792301,759
7,7,-1,2000,0.297927,120,8,10000,149,5,0,...,NaN,NaN,NaN,NaN,NaN,dart,0.219074,0.903463,0.790112,823
0,0,-1,12000,0.036808,2040,4,300,157,90,85,...,NaN,NaN,NaN,NaN,NaN,goss,0.218847,0.903561,0.789976,1464
5,5,-1,2000,0.142550,2800,3,600,224,30,45,...,NaN,NaN,NaN,NaN,NaN,dart,0.219338,0.902938,0.789967,394
10,10,-1,2000,0.206498,820,10,7600,109,65,25,...,NaN,NaN,NaN,NaN,NaN,dart,0.219434,0.903115,0.789697,489
3,3,-1,12000,0.155771,1440,3,5900,251,40,80,...,NaN,NaN,NaN,NaN,NaN,gbdt,0.219392,0.903197,0.789396,103


In [20]:
best_idx = np.argmax(df_params["amex"])
best_idx

2

In [21]:
#df_train = df_train.reset_index(drop=True)
#df_train

In [22]:
#optuna_parameters

In [23]:
optuna_params = df_params.iloc[best_idx]
optuna_params = dict(optuna_params)
optuna_params

{'Unnamed: 0': 2,
 'verbosity': -1,
 'n_estimators': 2000,
 'learning_rate': 0.2522936172843501,
 'num_leaves': 620,
 'max_depth': 10,
 'min_data_in_leaf': 7500,
 'max_bin': 146,
 'lambda_l1': 25,
 'lambda_l2': 0,
 'min_gain_to_split': 2.7806868606837614,
 'bagging_fraction': 0.8500000000000001,
 'bagging_freq': 5.0,
 'feature_fraction': 0.8,
 'subsample': nan,
 'reg_alpha': nan,
 'reg_lambda': nan,
 'min_sum_hessian_in_leaf': nan,
 'min_child_samples': nan,
 'colsample_bytree': nan,
 'subsample_freq': nan,
 'min_split_gain': nan,
 'boosting_type': 'dart',
 'loss': 0.2165227464374611,
 'accuracy': 0.9046180865264162,
 'amex': 0.7932202487793407,
 'time': 689}

In [24]:
df_params = pd.read_csv("optimizer_results_lgbm_lagged.csv")

# Clean Up Stored Optmizer Best Params for Use

Because the code above sometimes adds a few params that were not used, the classifier might complain

In [25]:
optuna_params = df_params.sort_values(by="amex", ascending=False).iloc[0]
optuna_params = dict(optuna_params)

keys = list(optuna_params.keys())
for param in keys:
    if isinstance(optuna_params[param], (np.number)) and np.isnan(optuna_params[param]):
        print(f"delete: {param}")
        del optuna_params[param]

if "bagging_freq" in optuna_params:
    optuna_params["bagging_freq"] = int(optuna_params["bagging_freq"])
del optuna_params["loss"]
del optuna_params["Unnamed: 0"]
del optuna_params["accuracy"]
del optuna_params["time"]
del optuna_params["amex"]


delete: subsample
delete: reg_alpha
delete: reg_lambda
delete: min_sum_hessian_in_leaf
delete: min_child_samples
delete: colsample_bytree
delete: subsample_freq
delete: min_split_gain


In [26]:
optuna_params

{'verbosity': -1,
 'n_estimators': 2000,
 'learning_rate': 0.2522936172843501,
 'num_leaves': 620,
 'max_depth': 10,
 'min_data_in_leaf': 7500,
 'max_bin': 146,
 'lambda_l1': 25,
 'lambda_l2': 0,
 'min_gain_to_split': 2.7806868606837614,
 'bagging_fraction': 0.8500000000000001,
 'bagging_freq': 5,
 'feature_fraction': 0.8,
 'boosting_type': 'dart'}

In [27]:
#dart is slow with large number of estimators (iterations), so limiting the max here
if optuna_params["boosting_type"] == "dart":
    optuna_params["n_estimators"] = 3000


# Run Actual Train and Predict Using Found Optimized Params

On the training data


In [28]:
#todo: max iterations 10k
#sometimes it is useful to run on smaller subset of data to get initial results faster
#final_models, final_preds = train_and_predict(optuna_params, df_full_train[:50000])
final_models, final_preds = train_and_predict(optuna_params, df_train, strat)


=== STARTING FOLD 1/5 ===
creating classifier
fitting


/usr/lib/python3.8/site-packages/lightgbm/callback.py:265: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


[100]	valid_0's binary_logloss: 0.224798
[200]	valid_0's binary_logloss: 0.220708
[300]	valid_0's binary_logloss: 0.218625
[400]	valid_0's binary_logloss: 0.217829
[500]	valid_0's binary_logloss: 0.217437
[600]	valid_0's binary_logloss: 0.217381
[700]	valid_0's binary_logloss: 0.217128
[800]	valid_0's binary_logloss: 0.217094
[900]	valid_0's binary_logloss: 0.216946
[1000]	valid_0's binary_logloss: 0.216867
[1100]	valid_0's binary_logloss: 0.216733
[1200]	valid_0's binary_logloss: 0.216645
[1300]	valid_0's binary_logloss: 0.216784
[1400]	valid_0's binary_logloss: 0.216734
[1500]	valid_0's binary_logloss: 0.216787
[1600]	valid_0's binary_logloss: 0.216793
[1700]	valid_0's binary_logloss: 0.216786
[1800]	valid_0's binary_logloss: 0.216809
[1900]	valid_0's binary_logloss: 0.216846
[2000]	valid_0's binary_logloss: 0.216877
[2100]	valid_0's binary_logloss: 0.216865
[2200]	valid_0's binary_logloss: 0.21688
[2300]	valid_0's binary_logloss: 0.21687
[2400]	valid_0's binary_logloss: 0.216802
[25

/usr/lib/python3.8/site-packages/lightgbm/callback.py:265: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


[100]	valid_0's binary_logloss: 0.224663
[200]	valid_0's binary_logloss: 0.220955
[300]	valid_0's binary_logloss: 0.219001
[400]	valid_0's binary_logloss: 0.218211
[500]	valid_0's binary_logloss: 0.218021
[600]	valid_0's binary_logloss: 0.217894
[700]	valid_0's binary_logloss: 0.217591
[800]	valid_0's binary_logloss: 0.217605
[900]	valid_0's binary_logloss: 0.217472
[1000]	valid_0's binary_logloss: 0.217343
[1100]	valid_0's binary_logloss: 0.217343
[1200]	valid_0's binary_logloss: 0.217322
[1300]	valid_0's binary_logloss: 0.217307
[1400]	valid_0's binary_logloss: 0.217222
[1500]	valid_0's binary_logloss: 0.217259
[1600]	valid_0's binary_logloss: 0.217215
[1700]	valid_0's binary_logloss: 0.217204
[1800]	valid_0's binary_logloss: 0.21734
[1900]	valid_0's binary_logloss: 0.217297
[2000]	valid_0's binary_logloss: 0.217342
[2100]	valid_0's binary_logloss: 0.217244
[2200]	valid_0's binary_logloss: 0.217137
[2300]	valid_0's binary_logloss: 0.217121
[2400]	valid_0's binary_logloss: 0.217062
[2

...

[100]	valid_0's binary_logloss: 0.222895
[200]	valid_0's binary_logloss: 0.218318
[300]	valid_0's binary_logloss: 0.216428
[400]	valid_0's binary_logloss: 0.21554
[500]	valid_0's binary_logloss: 0.215202
[600]	valid_0's binary_logloss: 0.215305
[700]	valid_0's binary_logloss: 0.215164
[800]	valid_0's binary_logloss: 0.215023
[900]	valid_0's binary_logloss: 0.214875
[1000]	valid_0's binary_logloss: 0.214677
[1100]	valid_0's binary_logloss: 0.214653
[1200]	valid_0's binary_logloss: 0.214551
[1300]	valid_0's binary_logloss: 0.214517
[1400]	valid_0's binary_logloss: 0.214438
[1500]	valid_0's binary_logloss: 0.214489
[1600]	valid_0's binary_logloss: 0.214366
[1700]	valid_0's binary_logloss: 0.214376
[1800]	valid_0's binary_logloss: 0.214385
[1900]	valid_0's binary_logloss: 0.214396
[2000]	valid_0's binary_logloss: 0.214355
[2100]	valid_0's binary_logloss: 0.214352
[2200]	valid_0's binary_logloss: 0.214318
[2300]	valid_0's binary_logloss: 0.214303
[2400]	valid_0's binary_logloss: 0.214359
[2

/usr/lib/python3.8/site-packages/lightgbm/callback.py:265: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


[100]	valid_0's binary_logloss: 0.223752
[200]	valid_0's binary_logloss: 0.219598
[300]	valid_0's binary_logloss: 0.21803
[400]	valid_0's binary_logloss: 0.217307
[500]	valid_0's binary_logloss: 0.216936
[600]	valid_0's binary_logloss: 0.216858
[700]	valid_0's binary_logloss: 0.216568
[800]	valid_0's binary_logloss: 0.216585
[900]	valid_0's binary_logloss: 0.216519
[1000]	valid_0's binary_logloss: 0.216373
[1100]	valid_0's binary_logloss: 0.216396
[1200]	valid_0's binary_logloss: 0.216521
[1300]	valid_0's binary_logloss: 0.216577
[1400]	valid_0's binary_logloss: 0.216491
[1500]	valid_0's binary_logloss: 0.216593
[1600]	valid_0's binary_logloss: 0.216528
[1700]	valid_0's binary_logloss: 0.216539
[1800]	valid_0's binary_logloss: 0.216577
[1900]	valid_0's binary_logloss: 0.216577
[2000]	valid_0's binary_logloss: 0.216517
[2100]	valid_0's binary_logloss: 0.216523
[2200]	valid_0's binary_logloss: 0.216519
[2300]	valid_0's binary_logloss: 0.216521
[2400]	valid_0's binary_logloss: 0.216541
[2

/usr/lib/python3.8/site-packages/lightgbm/callback.py:265: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


[100]	valid_0's binary_logloss: 0.226351
[200]	valid_0's binary_logloss: 0.222044
[300]	valid_0's binary_logloss: 0.219996
[400]	valid_0's binary_logloss: 0.219142
[500]	valid_0's binary_logloss: 0.218923
[600]	valid_0's binary_logloss: 0.218881
[700]	valid_0's binary_logloss: 0.218575
[800]	valid_0's binary_logloss: 0.218607
[900]	valid_0's binary_logloss: 0.218561
[1000]	valid_0's binary_logloss: 0.218532
[1100]	valid_0's binary_logloss: 0.21847
[1200]	valid_0's binary_logloss: 0.218425
[1300]	valid_0's binary_logloss: 0.218373
[1400]	valid_0's binary_logloss: 0.218263
[1500]	valid_0's binary_logloss: 0.218174
[1600]	valid_0's binary_logloss: 0.218194
[1700]	valid_0's binary_logloss: 0.218116
[1800]	valid_0's binary_logloss: 0.218121
[1900]	valid_0's binary_logloss: 0.218039
[2000]	valid_0's binary_logloss: 0.218038
[2100]	valid_0's binary_logloss: 0.217989
[2200]	valid_0's binary_logloss: 0.218035
[2300]	valid_0's binary_logloss: 0.21801
[2400]	valid_0's binary_logloss: 0.218046
[25

In [29]:
df_preds = pd.DataFrame()
df_preds["customer_ID"] = df_train["customer_ID"]
df_preds["prediction"] = final_preds

In [30]:
amex_metric_mod(y, final_preds[:, 0])    

0.793689046981534

In [31]:
df_preds.to_csv("predictions_lgbm_lagged.csv")

# Predict the Kaggle Test Set

In [32]:
#X = df_test.drop(["customer_ID", "S_2"], axis=1)
X = df_test.drop("customer_ID", axis=1)

In [33]:
from tqdm import tqdm

n_classes = 1
sub_preds = np.zeros((df_test.shape[0], n_classes))

#tqdm.auto()
tqdm.pandas()

for model in tqdm(final_models): #lqtm
    #could define num_iteration but default values should cover it
    #https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html#lightgbm.LGBMClassifier.predict_proba
    preds = model.predict_proba(X)
    preds_true = preds[:, 1]
    preds_true.shape = (X.shape[0], 1)
    sub_preds += preds_true
sub_preds /= N_FOLDS


100%|████████████████████████████████████████████████████████████████████| 5/5 [01:59<00:00, 23.81s/it]


In [34]:
submission = pd.DataFrame()
submission["customer_ID"] = df_test["customer_ID"]
submission["prediction"] = sub_preds
submission

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.021787
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.000869
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.052829
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.196066
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.862293
...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.008613
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.772294
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0.556056
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0.251052


In [35]:
submission.to_csv("submission_lgbm_lagged_seed2.csv", index=False)

In [36]:
#0.794